In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import math
from comtrade import Comtrade
import plotly.graph_objects as go
import plotly.express as px
plt.rcParams['savefig.directory'] = os.getcwd()

# change this to match the exported COMTRADE file
root = '../PSCAD/COMTRADEtest/playback'

# load all the analog channels into a dictionary of numpy arrays
# 1..3 = Va..Vc
# 4..6 = Ia..Ic
# 7 = Vrms
# 8 = P
# 9 = Q
# load all the analog channels into a dictionary of numpy arrays
rec = Comtrade ()
rec.load (root + '.cfg', root + '.dat')
t = np.array(rec.time)
channels = {}
for i in range(rec.analog_count):
    lbl = rec.analog_channel_ids[i]
    channels[lbl] = np.array (rec.analog[i])


In [ ]:
kVLNbase = 138.0 / math.sqrt(3.0)
MVAbase = 100.0
kAbase = MVAbase / kVLNbase / 3.0

def scale_factor(lbl):
  if 'P' in lbl:
    return 1.0 / MVAbase
  elif 'Q' in lbl:
    return 1.0 / MVAbase
  elif 'I' in lbl:
    return 1.0 / kAbase / math.sqrt(2.0)
  elif 'rms' in lbl:
    return 1.0
  elif 'V' in lbl:
    return 1.0 / kVLNbase / math.sqrt(2.0)
  return 1.0


class ColorList:
    def __init__(self):
        self.cidx = 0
        self.colors = px.colors.qualitative.Plotly
        self.n = len(self.colors)
    def __call__(self):
        return self.colors[self.cidx]
    def step(self):
        self.cidx = (self.cidx + 1) % self.n
    def setidx(self, cidx):
        self.cidx = cidx
    def getidx(self):
        return self.cidx
    def shiftidx(self, shift):
        self.cidx = (self.cidx + shift) % self.n

In [ ]:
print ('Channel           Min        Max       Mean')
for key, val in channels.items():
    print ('{:10s} {:10.4f} {:10.4f} {:10.4f}'.format (key, np.min(val), np.max(val), np.mean(val)))

Run the following cell to generate the plot using [Plotly](https://plotly.com/python/getting-started/) an interactive plotting library.

In [ ]:
fig = go.Figure().set_subplots(3, 1, shared_xaxes=True, x_title="Time [s]")
labels = [lbl for lbl in channels]
colors = ColorList()
for i in range(3):
    if i < 2:
        colors.setidx(0)
    for j in range(3):
        label = labels[3*i + j]
        fig.add_trace(go.Scatter(x=t, y=scale_factor(label)*channels[label],
                                 name=label, line_color=colors()),
                                 row=i+1, col=1)
        colors.step()
fig.update_layout(title='Case: ' + root, template="simple_white", width=1200, height=1000)
fig.update_xaxes(showgrid=True, minor_showgrid=True)
fig.update_yaxes(showgrid=True, minor_showgrid=True)
#### Display/Save
## uncomment the line below to save the plot to an html file (still interactive) that can be opened later
# fig.write_html(os.path.join(os.getcwd(), "comtrade_plot_test.html")) 
fig.show()

Run the following cell to generate the plots using [Matplotlib](https://matplotlib.org/)

In [ ]:
# display the channels in groups of three
# %matplotlib inline
%matplotlib notebook
# %matplotlib widget
labels = [lbl for lbl in channels]
fig, ax = plt.subplots(3, 1, sharex = 'col', figsize=(12,10), constrained_layout=True)
fig.suptitle ('Case: ' + root)
for i in range(3):
    for j in range(3):
        lbl = labels[3*i + j]
        ax[i].plot(t, scale_factor(lbl) * channels[lbl], label=lbl)
    ax[i].grid()
    ax[i].legend()

plt.show()